<img src="Figures/top_ML.png" alt="Drawing" style="width: 1000px;"/>

# "Digitalización de una carga doméstica"

## Postítulo "Digital Energy", Máster Smart Energy

## Módulo: Digitalización de la Energía

## Alumno: Max Antonio Missene Montecinos

### Objetivo:

#### Monitoreo de una carga doméstica mediante un procesador de datos construido en base a un microcontrolador Arduino.

### Pautas generales:

1. Descripción de la carga seleccionada para nuestro Proyecto. Modelo, potencia nominal, uso habitual que hacéis de ella, etc.
2. Descripción de vuestro montaje: Fotos del montaje con la carga, explicación breve del código para la realización de la medición.
3. Análisis preliminar de consumo de la carga: Captura del a corriente instantánea, grabar datos de corriente en RMS y realizar alguna gráfica (Excel mismo) y comentar el consumo observado.

## Generales

#### El primer paso que se realizará en el desarrollo del presente trabajo, será importar las librerías y funciones de librerías que utilizaremos a lo largo del código.

In [1]:
# Librerías usadas

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
from datetime import date, time, datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

#### A continuación, se procederá a importar la base de datos.

In [2]:
# Generar DataFrame (DF) a partir de base de datos de excel
columns=['Fecha','Corriente','Potencia']
BD_carga = pd.read_csv('Data/data_1000_C_20h.csv', names=columns)

### 1. Entendimiento de los datos, estadística descriptiva y visualizaciones

#### En primera instancia, se determinará el número de dimensiones que posee la base de datos.

In [3]:
# Se procederá a conocer las dimensiones del DF con la función "shape".
BD_carga.shape

(248, 3)

#### Es posible notar que el DataFrame contine 3 columnas y 248 filas. 

#### Así, se desplegaran las primeras filas del dataframe, para examinar de forma preliminar los datos.

In [4]:
# Desplegar primeras 5 filas del DF a partir de la función "head".
BD_carga.head()

,Fecha,Corriente,Potencia
0,2022-06-07 20:31:46.393620,0.22,50.49
1,2022-06-07 20:31:47.396596,0.21,47.28
2,2022-06-07 20:31:48.400759,0.22,49.72
3,2022-06-07 20:31:49.400014,0.22,51.09
4,2022-06-07 20:31:50.403693,0.21,48.74


#### De manera análoga, se desplegaran las primeras últimas filas del dataframe, para examinar de forma preliminar los datos.

In [5]:
# Desplegar última 5 filas del DF a partir de la función "tail".
BD_carga.tail()

,Fecha,Corriente,Potencia
243,2022-06-07 20:35:49.937707,0.22,49.46
244,2022-06-07 20:35:50.940709,0.21,48.24
245,2022-06-07 20:35:51.943865,0.22,50.15
246,2022-06-07 20:35:52.943749,0.22,50.07
247,2022-06-07 20:35:53.946841,0.22,49.55


In [6]:
# Desplegar última 5 filas del DF a partir de la función "tail".
BD_carga.dtypes

Fecha         object
Corriente    float64
Potencia     float64
dtype: object

#### Ya pudiendo mirar el contenido del dataframe de manera general, se realizará el cálculo de valores estadísticos de sus columnas.

In [7]:
# Desplegar última 5 filas del DF a partir de la función "tail".
BD_carga['Fecha']=pd.to_datetime(BD_carga['Fecha'])
BD_carga.dtypes


Fecha        datetime64[ns]
Corriente           float64
Potencia            float64
dtype: object

In [8]:
# Se aplica la función "describe" al dataframe, el cual calculará algunos parámetros estadísticos a cada columna.
BD_carga.describe()

,Corriente,Potencia
count,248.000000,248.000000
mean,6.071815,1396.411008
std,1.276556,293.655071
min,0.210000,47.280000
25%,6.340000,1458.820000
50%,6.350000,1460.940000
75%,6.360000,1462.822500
max,6.530000,1501.270000


In [9]:
BD_carga.isna().sum()

Fecha        0
Corriente    0
Potencia     0
dtype: int64

In [11]:
BD_carga['Energy_kWh']=0.0
BD_carga['time_dif']=0.0
BD_carga['cum_energy_kwh']=0.0
longitud=len(BD_carga['Energy_kWh'])

for x in range(0, longitud):
 if BD_carga['Corriente'][x]<0.0:
        BD_carga.at[x, 'Corriente']=0.0
 if BD_carga['Potencia'][x]<0.0:
        BD_carga.at[x, 'Potencia']=0.0
for x in range(1, longitud):
    BD_carga.at[x, 'Energy_kWh']=BD_carga['Potencia'][x]*((BD_carga['Fecha'][x]-BD_carga['Fecha'][x-1]).total_seconds()/3600000)
    BD_carga.at[x, 'time_dif']=(BD_carga['Fecha'][x]-BD_carga['Fecha'][x-1]).total_seconds()
    BD_carga.at[x, 'cum_energy_kwh']=BD_carga['cum_energy_kwh'][x-1]+BD_carga['Energy_kWh'][x]
    

In [12]:
BD_carga.head()

,Fecha,Corriente,Potencia,Energy_kWh,time_dif,cum_energy_kwh
0,2022-06-07 20:31:46.393620,0.22,50.49,0.000000,0.000000,0.000000
1,2022-06-07 20:31:47.396596,0.21,47.28,0.000013,1.002976,0.000013
2,2022-06-07 20:31:48.400759,0.22,49.72,0.000014,1.004163,0.000027
3,2022-06-07 20:31:49.400014,0.22,51.09,0.000014,0.999255,0.000041
4,2022-06-07 20:31:50.403693,0.21,48.74,0.000014,1.003679,0.000055


In [13]:
BD_carga.tail()

,Fecha,Corriente,Potencia,Energy_kWh,time_dif,cum_energy_kwh
243,2022-06-07 20:35:49.937707,0.22,49.46,0.000014,0.999635,0.096343
244,2022-06-07 20:35:50.940709,0.21,48.24,0.000013,1.003002,0.096357
245,2022-06-07 20:35:51.943865,0.22,50.15,0.000014,1.003156,0.096371
246,2022-06-07 20:35:52.943749,0.22,50.07,0.000014,0.999884,0.096385
247,2022-06-07 20:35:53.946841,0.22,49.55,0.000014,1.003092,0.096398


In [14]:
BD_carga['Energy_kWh'].sum()

0.09639833886305554

In [15]:
((BD_carga['time_dif'].sum())/60)/60

0.0687647836111111

In [18]:
# conda install -c plotly plotly
import plotly.offline as py;
from plotly import tools;
import plotly.graph_objs as go;
from ipywidgets import widgets;
from ipywidgets import interact;
import warnings
warnings.filterwarnings('ignore')
py.init_notebook_mode(connected= True)

In [20]:
fig = go.Figure(
    data=[
        go.Line(x=BD_carga['Fecha'][0:12*75],
                y=BD_carga['Potencia'][0:12*75], name="Power")
    ],
    layout=go.Layout(
        title=dict(text='Potencia RMS Hervidor Eléctrico: 20h;AguaNormal;1000ml'),
        xaxis=dict(title='Hora'),
        yaxis=dict(title='Potencia [W]')
    )
)
fig.show()
        
        